In [1]:

# Step 1: Compute Manhattan lane miles by NTA (exposure)
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from pathlib import Path

PROCESSED = Path.cwd().parent / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

def fetch_json(url, params):
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    return r.json()

# Fetch bus lanes (Manhattan) with geometry intact
lanes = fetch_json(
    "https://data.cityofnewyork.us/resource/ycrg-ses3.json",
    {"$where": "upper(boro) like '%MAN%'", "$limit": 50000}
)
lanes_df = pd.DataFrame(lanes)
lanes_gdf = gpd.GeoDataFrame(
    lanes_df,
    geometry=[shape(g) if isinstance(g, dict) else None for g in lanes_df.get("the_geom", [])],
    crs="EPSG:4326"
)
# Keep true bus lanes if present
if "lane_type1" in lanes_gdf.columns:
    lanes_gdf = lanes_gdf[lanes_gdf["lane_type1"].str.contains("Bus Lane", na=False)]

# Fetch NTAs (Manhattan) with geometry
ntas = fetch_json(
    "https://data.cityofnewyork.us/resource/9nt8-h7nd.json",
    {"$where": "borocode=1", "$limit": 50000}
)
ntas_df = pd.DataFrame(ntas)
ntas_gdf = gpd.GeoDataFrame(
    ntas_df,
    geometry=[shape(g) if isinstance(g, dict) else None for g in ntas_df.get("the_geom", [])],
    crs="EPSG:4326"
)

# Project to feet (NY State Plane) and compute length by NTA
lanes_2263 = lanes_gdf.to_crs(2263)
ntas_2263 = ntas_gdf.to_crs(2263)

# Clip lines to polygons, then sum lengths per NTA
clipped = gpd.overlay(lanes_2263, ntas_2263[["nta2020", "ntaname", "geometry"]], how="intersection")
clipped["length_miles"] = clipped.length / 5280.0
lane_miles = (
    clipped.groupby(["nta2020", "ntaname"], as_index=False)["length_miles"].sum()
    .sort_values("length_miles", ascending=False)
)

out = PROCESSED / "lane_miles_by_nta_manhattan.csv"
lane_miles.to_csv(out, index=False)
print(f"Saved lane miles by NTA → {out} (NTAs: {lane_miles.shape[0]}, total miles: {lane_miles['length_miles'].sum():.1f})")
print(lane_miles.head(5))


Saved lane miles by NTA → /Users/mohamedhiba/Fall 2025/datathon/data/processed/lane_miles_by_nta_manhattan.csv (NTAs: 23, total miles: 35.2)
   nta2020                                      ntaname  length_miles
10  MN0603                         Murray Hill-Kips Bay      4.410495
3   MN0303                                 East Village      2.960079
14  MN0801  Upper East Side-Lenox Hill-Roosevelt Island      2.718747
6   MN0501          Midtown South-Flatiron-Union Square      2.591855
20  MN1102                          East Harlem (North)      2.442016


In [2]:
# Step 2: Join ACE → NTA, aggregate by NTA × hour, compute rates
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
from pathlib import Path #just to get the path and navigate through the data 
import requests

DATA = Path.cwd().parent / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

# Load lane miles (from Step 1)
lane_miles = pd.read_csv(PROCESSED / "lane_miles_by_nta_manhattan.csv")
lane_miles = lane_miles.rename(columns={"length_miles": "lane_miles"})

# Load ACE violations (Manhattan slice)
ace_path = RAW / "ace_violations_manhattan.csv"
ace_df = pd.read_csv(ace_path)
if ace_df.empty:
    raise RuntimeError("ACE file is empty. Run the fetch notebook cell first.")

# Extract coordinates, build GeoDataFrame
ace_df["violation_latitude"] = pd.to_numeric(ace_df.get("violation_latitude"), errors="coerce")
ace_df["violation_longitude"] = pd.to_numeric(ace_df.get("violation_longitude"), errors="coerce")
ace_points = ace_df.dropna(subset=["violation_latitude", "violation_longitude"]).copy()
ace_gdf = gpd.GeoDataFrame(
    ace_points,
    geometry=[Point(xy) for xy in zip(ace_points["violation_longitude"], ace_points["violation_latitude"])],
    crs="EPSG:4326"
).to_crs(2263)

# Hour of day from first_occurrence
ace_gdf["first_occurrence"] = pd.to_datetime(ace_gdf["first_occurrence"], errors="coerce")
ace_gdf["hour"] = ace_gdf["first_occurrence"].dt.hour

# Fetch NTA geometry (Manhattan) for spatial join
r = requests.get(
    "https://data.cityofnewyork.us/resource/9nt8-h7nd.json",
    params={"$where": "borocode=1", "$limit": 50000}, timeout=60
)
r.raise_for_status()
ntas_df = pd.DataFrame(r.json())
ntas_gdf = gpd.GeoDataFrame(
    ntas_df,
    geometry=[shape(g) if isinstance(g, dict) else None for g in ntas_df.get("the_geom", [])],
    crs="EPSG:4326"
).to_crs(2263)

# Spatial join points → polygons
joined = gpd.sjoin(ace_gdf, ntas_gdf[["nta2020", "ntaname", "geometry"]], how="inner", predicate="within")

# Aggregate violations by NTA × hour
viol_by_nta_hour = (
    joined.groupby(["nta2020", "ntaname", "hour"], as_index=False).size()
    .rename(columns={"size": "violations"})
)

# Compute rates
rates = viol_by_nta_hour.merge(lane_miles, on=["nta2020", "ntaname"], how="left")
rates = rates[rates["lane_miles"] > 0]
rates["rate"] = rates["violations"] / rates["lane_miles"]

# Save outputs
viol_by_nta_hour.to_csv(PROCESSED / "violations_by_nta_hour_manhattan.csv", index=False)
rates.to_csv(PROCESSED / "rates_by_nta_hour_manhattan.csv", index=False)

print("Rows:", len(rates), "NTAs:", rates['nta2020'].nunique())
print(rates.sort_values("rate", ascending=False).head(8)[["nta2020", "hour", "rate"]])


Rows: 551 NTAs: 23
    nta2020  hour          rate
735  MN1203    16  24589.580432
396  MN0702    13  21602.554201
514  MN0901    11  20786.494974
397  MN0702    14  19809.689159
513  MN0901    10  19743.361028
394  MN0702    11  19310.038245
515  MN0901    12  19145.610116
395  MN0702    12  18146.145529


In [ ]:
# Step 3a: Proximity to schools/hospitals (100 m), hourly spikes
import json, ast, math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import matplotlib.pyplot as plt

DATA = Path.cwd().parent / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

FT_PER_M = 3.28084
BUFFER_FT = 100 * FT_PER_M  # 100 meters in feet (EPSG:2263 units)

# 1) Load ACE Manhattan and build GeoDataFrame with hour
ace_df = pd.read_csv(RAW / "ace_violations_manhattan.csv")
ace_df["violation_latitude"] = pd.to_numeric(ace_df.get("violation_latitude"), errors="coerce")
ace_df["violation_longitude"] = pd.to_numeric(ace_df.get("violation_longitude"), errors="coerce")
ace_points = ace_df.dropna(subset=["violation_latitude", "violation_longitude"]).copy()
ace_gdf = gpd.GeoDataFrame(
    ace_points,
    geometry=[Point(xy) for xy in zip(ace_points["violation_longitude"], ace_points["violation_latitude"])],
    crs="EPSG:4326"
).to_crs(2263)
ace_gdf["first_occurrence"] = pd.to_datetime(ace_gdf["first_occurrence"], errors="coerce")
ace_gdf["hour"] = ace_gdf["first_occurrence"].dt.hour.fillna(0).astype(int)

# 2) Load schools (latitude/longitude + school_name)
schools_path = RAW / "schools_manhattan.csv"
schools_df = pd.read_csv(schools_path)
lat_col = next((c for c in ["latitude","LATITUDE","lat","Lat"] if c in schools_df.columns), None)
lon_col = next((c for c in ["longitude","LONGITUDE","lon","Lon"] if c in schools_df.columns), None)
name_col = "school_name" if "school_name" in schools_df.columns else None
if not lat_col or not lon_col:
    raise RuntimeError("schools_manhattan.csv is missing latitude/longitude columns.")
schools_gdf = gpd.GeoDataFrame(
    schools_df.dropna(subset=[lat_col, lon_col]).copy(),
    geometry=gpd.points_from_xy(schools_df[lon_col], schools_df[lat_col], crs="EPSG:4326")
).to_crs(2263)
schools_buf = schools_gdf.buffer(BUFFER_FT)
schools_buf_gdf = gpd.GeoDataFrame(schools_gdf[[name_col]] if name_col else schools_gdf[[lat_col,lon_col]], geometry=schools_buf, crs=2263)

# 3) Load hospitals (location_1 point + facility_name)
hosp_path = RAW / "hospitals_manhattan.csv"
hosp_df = pd.read_csv(hosp_path)
# Parse Socrata 'location_1' column into lon/lat
lon_list, lat_list = [], []
for v in hosp_df.get("location_1", []):
    if isinstance(v, str):
        try:
            d = ast.literal_eval(v)
        except Exception:
            d = None
    elif isinstance(v, dict):
        d = v
    else:
        d = None
    if isinstance(d, dict) and isinstance(d.get("coordinates"), (list, tuple)) and len(d["coordinates"])==2:
        lon_list.append(d["coordinates"][0]); lat_list.append(d["coordinates"][1])
    else:
        lon_list.append(math.nan); lat_list.append(math.nan)

hosp_df["_lon"] = pd.to_numeric(lon_list, errors="coerce")
hosp_df["_lat"] = pd.to_numeric(lat_list, errors="coerce")
name_hosp_col = "facility_name" if "facility_name" in hosp_df.columns else None
hosp_gdf = gpd.GeoDataFrame(
    hosp_df.dropna(subset=["_lat","_lon"]).copy(),
    geometry=gpd.points_from_xy(hosp_df["_lon"], hosp_df["_lat"], crs="EPSG:4326")
).to_crs(2263)
hosp_buf = hosp_gdf.buffer(BUFFER_FT)
hosp_buf_gdf = gpd.GeoDataFrame(hosp_gdf[[name_hosp_col]] if name_hosp_col else hosp_gdf[["_lat","_lon"]], geometry=hosp_buf, crs=2263)

# 4) Flag proximity via spatial join (within)
ace_with_school = gpd.sjoin(ace_gdf, schools_buf_gdf, how="left", predicate="within")
ace_with_school["near_school"] = ace_with_school.index_right.notna()
ace_with_hosp = gpd.sjoin(ace_gdf, hosp_buf_gdf, how="left", predicate="within")
ace_with_hosp["near_hospital"] = ace_with_hosp.index_right.notna()

# Merge flags into one table aligned on ACE index
prox = ace_gdf[["hour"]].copy()
prox = prox.join(ace_with_school["near_school"], how="left")
prox = prox.join(ace_with_hosp["near_hospital"], how="left", rsuffix="_h")
prox["near_school"] = prox["near_school"].fillna(False)
prox["near_hospital"] = prox["near_hospital"].fillna(False)

# 5) Aggregate by hour into windows
school_windows = {
    "school_7_9": [7,8,9],
    "school_14_16": [14,15,16]
}
hosp_windows = {
    "hosp_6_8": [6,7,8],
    "hosp_14_16": [14,15,16],
    "hosp_22_23": [22,23]
}

# schools
sch_hour = prox.groupby(["hour","near_school"], as_index=False).size().rename(columns={"size":"count"})
sch_hour.to_csv(PROCESSED / "proximity_schools_by_hour_mn.csv", index=False)

# hospitals
hosp_hour = prox.groupby(["hour","near_hospital"], as_index=False).size().rename(columns={"size":"count"})
hosp_hour.to_csv(PROCESSED / "proximity_hospitals_by_hour_mn.csv", index=False)

# 6) Quick bar charts
plt.figure(figsize=(9,4))
plot_df = sch_hour.pivot(index="hour", columns="near_school", values="count").fillna(0)
plot_df.rename(columns={True:"Near schools", False:"Other"}, inplace=True)
plot_df.plot(kind="bar", ax=plt.gca(), color=["#1f77b4","#999999"])
plt.title("ACE violations by hour — near schools")
plt.xlabel("Hour of day"); plt.ylabel("Count")
plt.tight_layout(); plt.savefig(PROCESSED / "proximity_schools_by_hour_mn.png", dpi=200, bbox_inches="tight"); plt.close()

plt.figure(figsize=(9,4))
plot_df = hosp_hour.pivot(index="hour", columns="near_hospital", values="count").fillna(0)
plot_df.rename(columns={True:"Near hospitals", False:"Other"}, inplace=True)
plot_df.plot(kind="bar", ax=plt.gca(), color=["#d62728","#999999"])
plt.title("ACE violations by hour — near hospitals")
plt.xlabel("Hour of day"); plt.ylabel("Count")
plt.tight_layout(); plt.savefig(PROCESSED / "proximity_hospitals_by_hour_mn.png", dpi=200, bbox_inches="tight"); plt.close()

print("Saved:",
      PROCESSED / "proximity_schools_by_hour_mn.csv",
      PROCESSED / "proximity_hospitals_by_hour_mn.csv",
      PROCESSED / "proximity_schools_by_hour_mn.png",
      PROCESSED / "proximity_hospitals_by_hour_mn.png")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_schools_by_hour_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_hospitals_by_hour_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_schools_by_hour_mn.png /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_hospitals_by_hour_mn.png


In [ ]:
# Step 3b: Hourly shares near schools/hospitals and top-hour tables
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

DATA = Path.cwd().parent / "data"
PROCESSED = DATA / "processed"

# Load the hourly counts (fall back to files to avoid state issues)
sch_hour = pd.read_csv(PROCESSED / "proximity_schools_by_hour_mn.csv")
hosp_hour = pd.read_csv(PROCESSED / "proximity_hospitals_by_hour_mn.csv")

# Compute shares per hour
sch_tot = sch_hour.groupby("hour", as_index=False)["count"].sum().rename(columns={"count":"total"})
sch_true = sch_hour[sch_hour["near_school"]==True][["hour","count"]].rename(columns={"count":"near"})
sch_share = sch_tot.merge(sch_true, on="hour", how="left").fillna({"near":0.0})
sch_share["share_near_school"] = sch_share["near"] / sch_share["total"].replace({0: pd.NA})

hosp_tot = hosp_hour.groupby("hour", as_index=False)["count"].sum().rename(columns={"count":"total"})
hosp_true = hosp_hour[hosp_hour["near_hospital"]==True][["hour","count"]].rename(columns={"count":"near"})
hosp_share = hosp_tot.merge(hosp_true, on="hour", how="left").fillna({"near":0.0})
hosp_share["share_near_hospital"] = hosp_share["near"] / hosp_share["total"].replace({0: pd.NA})

# Save CSVs
sch_share.to_csv(PROCESSED / "proximity_schools_hourly_share_mn.csv", index=False) 
hosp_share.to_csv(PROCESSED / "proximity_hospitals_hourly_share_mn.csv", index=False)

# Top hours tables
sch_top = sch_share.sort_values("share_near_school", ascending=False).head(5)
hosp_top = hosp_share.sort_values("share_near_hospital", ascending=False).head(5)
sch_top.to_csv(PROCESSED / "proximity_schools_top_hours_mn.csv", index=False)
hosp_top.to_csv(PROCESSED / "proximity_hospitals_top_hours_mn.csv", index=False)

# Plots
plt.figure(figsize=(9,4))
plt.plot(sch_share["hour"], sch_share["share_near_school"], marker="o", color="#1f77b4")
plt.title("Share of ACE violations near schools by hour (Manhattan)")
plt.xlabel("Hour of day"); plt.ylabel("Share near schools")
plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(PROCESSED / "proximity_schools_hourly_share_mn.png", dpi=200, bbox_inches="tight"); plt.close()

plt.figure(figsize=(9,4))
plt.plot(hosp_share["hour"], hosp_share["share_near_hospital"], marker="o", color="#d62728")
plt.title("Share of ACE violations near hospitals by hour (Manhattan)")
plt.xlabel("Hour of day"); plt.ylabel("Share near hospitals")
plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(PROCESSED / "proximity_hospitals_hourly_share_mn.png", dpi=200, bbox_inches="tight"); plt.close()

print("Saved:",
      PROCESSED / "proximity_schools_hourly_share_mn.csv",
      PROCESSED / "proximity_hospitals_hourly_share_mn.csv",
      PROCESSED / "proximity_schools_top_hours_mn.csv",
      PROCESSED / "proximity_hospitals_top_hours_mn.csv",
      PROCESSED / "proximity_schools_hourly_share_mn.png",
      PROCESSED / "proximity_hospitals_hourly_share_mn.png")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_schools_hourly_share_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_hospitals_hourly_share_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_schools_top_hours_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_hospitals_top_hours_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_schools_hourly_share_mn.png /Users/mohamedhiba/Fall 2025/datathon/data/processed/proximity_hospitals_hourly_share_mn.png


In [9]:
# Step 4: Equity prep — aggregate rates to CDTA via NTA→CDTA crosswalk
import pandas as pd
import requests
from pathlib import Path

DATA = Path.cwd().parent / "data"
PROCESSED = DATA / "processed"
RAW = DATA / "raw"

# Load rates by NTA × hour (has lane_miles)
rates_nta_hr = pd.read_csv(PROCESSED / "rates_by_nta_hour_manhattan.csv")
assert {"nta2020","hour","rate","lane_miles"}.issubset(rates_nta_hr.columns)

# Fetch tract→NTA/CDTA crosswalk and build NTA→CDTA mapping (majority by tract count)
r = requests.get(
    "https://data.cityofnewyork.us/resource/hm78-6dwm.json",
    params={"$where": "borocode=1", "$select": "ntacode,cdtacode,count(geoid) as n", "$group": "ntacode,cdtacode"},
    timeout=60
)
r.raise_for_status()
xwalk = pd.DataFrame(r.json())
# Ensure proper types
xwalk["n"] = pd.to_numeric(xwalk["n"], errors="coerce")
# Choose CDTA with max tract count per NTA
nta_to_cdta = (
    xwalk.sort_values(["ntacode","n"], ascending=[True, False])
         .drop_duplicates(subset=["ntacode"])[["ntacode","cdtacode"]]
         .rename(columns={"ntacode":"nta2020"})
)

# Merge mapping
rates_nta_hr_map = rates_nta_hr.merge(nta_to_cdta, on="nta2020", how="left")

# Weighted aggregation to CDTA × hour (weights = lane_miles)
def weighted_mean(df, val="rate", w="lane_miles"):
    d = df[[val, w]].dropna()
    denom = d[w].sum()
    return (d[val]*d[w]).sum()/denom if denom else float("nan")

agg = (
    rates_nta_hr_map.groupby(["cdtacode","hour"], as_index=False)
    .apply(lambda g: pd.Series({
        "lane_miles": g["lane_miles"].sum(),
        "rate_wm": weighted_mean(g, "rate", "lane_miles")
    }))
)

# Also compute CDTA overall mean rate (weighted across hours by lane miles)
cdta_overall = (
    agg.groupby("cdtacode", as_index=False)
       .apply(lambda g: pd.Series({
           "lane_miles": g["lane_miles"].mean(),
           "rate_wm_overall": g["rate_wm"].mean()
       }))
)

# Save
agg.to_csv(PROCESSED / "rates_by_cdta_hour_manhattan.csv", index=False)
cdta_overall.to_csv(PROCESSED / "rates_by_cdta_overall_manhattan.csv", index=False)

print("Saved:",
      PROCESSED / "rates_by_cdta_hour_manhattan.csv",
      PROCESSED / "rates_by_cdta_overall_manhattan.csv")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/rates_by_cdta_hour_manhattan.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/rates_by_cdta_overall_manhattan.csv


/var/folders/sw/nd422j2x1hd_b_t5p52w2_640000gn/T/ipykernel_27682/4074089522.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
/var/folders/sw/nd422j2x1hd_b_t5p52w2_640000gn/T/ipykernel_27682/4074089522.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [12]:
# Step 5a (auto): Fetch ACS tract income and aggregate to CDTA (creates ccc_income_cdta.csv)
import pandas as pd
import requests
from pathlib import Path

DATA = Path.cwd().parent / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

# ACS variables: B19013_001E (Median household income), B11001_001E (Total households)
YEAR = 2022
BASE = f"https://api.census.gov/data/{YEAR}/acs/acs5"
VARS = ["NAME","B19013_001E","B11001_001E"]
NY_COUNTIES = {"005":"Bronx","047":"Kings","061":"New York","081":"Queens","085":"Richmond"}

rows = []
for fips in NY_COUNTIES.keys():
    params = {
        "get": ",".join(VARS),
        "for": "tract:*",
        "in": f"state:36 county:{fips}"
    }
    r = requests.get(BASE, params=params, timeout=60); r.raise_for_status()
    data = r.json()
    cols = data[0]
    for d in data[1:]:
        rec = dict(zip(cols, d))
        rows.append(rec)
acs = pd.DataFrame(rows)

# Build GEOID
acs["state"] = acs["state"].astype(str).str.zfill(2)
acs["county"] = acs["county"].astype(str).str.zfill(3)
acs["tract"] = acs["tract"].astype(str).str.zfill(6)
acs["geoid"] = acs["state"] + acs["county"] + acs["tract"]
acs["income_median"] = pd.to_numeric(acs["B19013_001E"], errors="coerce")
acs["households"] = pd.to_numeric(acs["B11001_001E"], errors="coerce")
acs = acs[["geoid","income_median","households"]]

# Map tracts → CDTA via hm78-6dwm (Manhattan only)
x = requests.get(
    "https://data.cityofnewyork.us/resource/hm78-6dwm.json",
    params={"$select":"geoid,cdtacode","$where":"geoid is not null AND cdtacode like 'MN%'"}, timeout=60
)
x.raise_for_status()
xwalk = pd.DataFrame(x.json())
tract_to_cdta = acs.merge(xwalk, on="geoid", how="inner")

# Clean income values: drop implausible or sentinel negatives, and zero/neg households
tract_to_cdta = tract_to_cdta[(tract_to_cdta["households"]>0)]
tract_to_cdta.loc[~tract_to_cdta["income_median"].between(10000, 300000), "income_median"] = pd.NA

# Aggregate to CDTA (household-weighted average of tract median incomes)
def weighted_mean(group):
    d = group[["income_median","households"]].dropna()
    if d.empty:
        return None
    w = d["households"]
    v = d["income_median"]
    denom = w.sum()
    return (v*w).sum()/denom if denom else None

cdta_income = (tract_to_cdta.groupby("cdtacode", as_index=False, group_keys=False)
               .apply(lambda g: pd.Series({
                   "median_household_income": weighted_mean(g)
               })))

out = RAW / "ccc_income_cdta.csv"
cdta_income.to_csv(out, index=False)
print(f"Saved CDTA income to {out} (rows: {len(cdta_income)})")


Saved CDTA income to /Users/mohamedhiba/Fall 2025/datathon/data/raw/ccc_income_cdta.csv (rows: 12)


/var/folders/sw/nd422j2x1hd_b_t5p52w2_640000gn/T/ipykernel_27682/2451136448.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [15]:
# Step 5: Equity — join CDTA rates to income and compute deciles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

DATA = Path.cwd().parent / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

rates_cdta = pd.read_csv(PROCESSED / "rates_by_cdta_overall_manhattan.csv")
rates_cdta["cdtacode"] = rates_cdta["cdtacode"].astype(str).str.upper()

income_path = RAW / "ccc_income_cdta.csv"
if not income_path.exists():
    # Create a template listing all CDTAs present in rates, to be filled once
    all_cdta = sorted(rates_cdta["cdtacode"].dropna().unique())
    tpl = pd.DataFrame({
        "cdtacode": all_cdta,
        "median_household_income": [np.nan]*len(all_cdta)
    })
    tpl.to_csv(income_path, index=False)
    raise FileNotFoundError(f"Missing {income_path}. A template with {len(all_cdta)} CDTAs was created; please fill median_household_income and rerun.")

income = pd.read_csv(income_path)
income = income.rename(columns={c: c.lower() for c in income.columns})
income["cdtacode"] = income["cdtacode"].astype(str).str.upper().str.strip()
income["median_household_income"] = pd.to_numeric(income["median_household_income"], errors="coerce")

# Merge
merged = rates_cdta.merge(income[["cdtacode","median_household_income"]], on="cdtacode", how="left")

# Drop rows without income
merged = merged.dropna(subset=["median_household_income"]).copy()

# Income deciles
merged["income_decile"] = pd.qcut(merged["median_household_income"], q=10, labels=False, duplicates="drop")

# Aggregate trend
trend = (merged.groupby("income_decile", as_index=False)["rate_wm_overall"].mean()
                  .rename(columns={"rate_wm_overall":"mean_rate"}))
trend.to_csv(PROCESSED / "equity_rates_by_income_decile_mn.csv", index=False)

# Also save top/bottom lists
top_cdta = merged.sort_values("rate_wm_overall", ascending=False)[["cdtacode","median_household_income","rate_wm_overall"]].head(10)
bot_cdta = merged.sort_values("rate_wm_overall", ascending=True)[["cdtacode","median_household_income","rate_wm_overall"]].head(10)
top_cdta.to_csv(PROCESSED / "equity_top_cdta_by_rate_mn.csv", index=False)
bot_cdta.to_csv(PROCESSED / "equity_bottom_cdta_by_rate_mn.csv", index=False)

# Plot trend
plt.figure(figsize=(7,4))
plt.plot(trend["income_decile"], trend["mean_rate"], marker="o", color="#444")
plt.title("Violation rate vs income decile (CDTA, Manhattan)")
plt.xlabel("Income decile (0=lowest)"); plt.ylabel("Mean rate (violations per lane-mile)")
plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(PROCESSED / "equity_rate_vs_income_decile_mn.png", dpi=200, bbox_inches="tight"); plt.close()

print("Saved:", PROCESSED / "equity_rates_by_income_decile_mn.csv",
      PROCESSED / "equity_top_cdta_by_rate_mn.csv",
      PROCESSED / "equity_bottom_cdta_by_rate_mn.csv",
      PROCESSED / "equity_rate_vs_income_decile_mn.png")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/equity_rates_by_income_decile_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/equity_top_cdta_by_rate_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/equity_bottom_cdta_by_rate_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/equity_rate_vs_income_decile_mn.png


In [1]:
# Step 6: "Where and when" — top NTA × hour by normalized rate
import pandas as pd
from pathlib import Path

DATA = Path.cwd().parent / "data"
PROCESSED = DATA / "processed"

rates = pd.read_csv(PROCESSED / "rates_by_nta_hour_manhattan.csv")
# Basic guardrails
rates = rates[(rates["lane_miles"] > 0.05)].copy()  # drop tiny denominators
rates = rates.dropna(subset=["rate"]) 

# Top overall windows
top_overall = (rates.sort_values("rate", ascending=False)
                    .head(20)[["nta2020","ntaname","hour","violations","lane_miles","rate"]])
top_overall.to_csv(PROCESSED / "where_when_top_overall_mn.csv", index=False)

# Top 5 per hour
tops = []
for h, grp in rates.groupby("hour"):
    g = grp.sort_values("rate", ascending=False).head(5)
    g = g[["nta2020","ntaname","hour","violations","lane_miles","rate"]]
    tops.append(g)
per_hour_top = pd.concat(tops, ignore_index=True)
per_hour_top.to_csv(PROCESSED / "where_when_top_per_hour_mn.csv", index=False)

print("Saved:", PROCESSED / "where_when_top_overall_mn.csv",
      PROCESSED / "where_when_top_per_hour_mn.csv")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/where_when_top_overall_mn.csv /Users/mohamedhiba/Fall 2025/datathon/data/processed/where_when_top_per_hour_mn.csv


In [2]:
# Step 6b: Turn "where/when" into a short action playbook
import pandas as pd
from pathlib import Path

DATA = Path.cwd().parent / "data"
PROCESSED = DATA / "processed"

# Load top overall NTA×hour by rate
df = pd.read_csv(PROCESSED / "where_when_top_overall_mn.csv")

# Tag time windows
def window_tag(h):
    h = int(h)
    if h in (7,8,9):
        return "school_AM (7–9)"
    if h in (14,15,16):
        return "school_PM (14–16)"
    if h in (6,7,8):
        return "hospital_AM (6–8)"
    if h in (22,23):
        return "hospital_late (22–23)"
    return "other"

df["window"] = df["hour"].apply(window_tag)

# Simple recommendation text
def recommend(row):
    w = row["window"]
    if w.startswith("school_AM") or w.startswith("school_PM"):
        return "Post-stop signed loading zone for drop-off/pick-up; targeted enforcement in window"
    if w.startswith("hospital"):
        return "Time-based loading curb and brief tow window aligned to shift times"
    return "Light protection or signage; monitor and deploy targeted enforcement"

df["recommendation"] = df.apply(recommend, axis=1)

# Clean columns and round
out = df[["nta2020","ntaname","hour","lane_miles","violations","rate","window","recommendation"]].copy()
out["rate"] = out["rate"].round(2)

# Keep top 12 for a 1-pager
playbook = out.head(12)
playbook.to_csv(PROCESSED / "where_when_action_playbook_mn.csv", index=False)
print("Saved:", PROCESSED / "where_when_action_playbook_mn.csv")


Saved: /Users/mohamedhiba/Fall 2025/datathon/data/processed/where_when_action_playbook_mn.csv
